# LSTM Trading AI for XAUUSD & BTCUSD

A comprehensive deep learning trading system using LSTM neural networks with attention mechanisms to predict and trade XAUUSD (Gold) and BTCUSD (Bitcoin).

This notebook provides an easy way to run the complete trading AI system without local installation issues.

## Setup and Installation

First, let's install all required dependencies and clone the repository.

In [ ]:
# Install system dependencies
!apt-get update && apt-get install -y git

# Clone the repository
!git clone https://github.com/JonusNattapong/Trading-Ai-Lstm.git
%cd Trading-Ai-Lstm

# Install Python dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -r requirements.txt

## Import Libraries and Test Setup

Let's verify that all libraries are installed correctly.

In [ ]:
import torch
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print("All libraries imported successfully!")

## Data Collection

Let's scrape historical data for XAUUSD (Gold) and BTCUSD (Bitcoin).

In [ ]:
# Import our data scraper
from data_scraper import DataScraper

# Initialize scraper
scraper = DataScraper()

# Scrape data for both pairs
print("Scraping XAUUSD data...")
xauusd_data = scraper.fetch_yahoo_data('GC=F', '2020-01-01', '2024-01-01')
print(f"XAUUSD data shape: {xauusd_data.shape}")

print("\nScraping BTCUSD data...")
btcusd_data = scraper.fetch_yahoo_data('BTC-USD', '2020-01-01', '2024-01-01')
print(f"BTCUSD data shape: {btcusd_data.shape}")

# Display sample data
print("\nXAUUSD Sample Data:")
print(xauusd_data.head())

print("\nBTCUSD Sample Data:")
print(btcusd_data.head())

## Data Preprocessing

Add technical indicators and prepare data for LSTM training.

In [ ]:
# Import preprocessing module
from data_preprocessing import DataPreprocessor

# Initialize preprocessor
preprocessor = DataPreprocessor()

# Process XAUUSD data
print("Processing XAUUSD data...")
xauusd_processed = preprocessor.prepare_features(xauusd_data)
print(f"XAUUSD processed shape: {xauusd_processed.shape}")
print(f"XAUUSD features: {len(preprocessor.feature_columns)}")

# Process BTCUSD data
print("\nProcessing BTCUSD data...")
btcusd_processed = preprocessor.prepare_features(btcusd_data)
print(f"BTCUSD processed shape: {btcusd_processed.shape}")
print(f"BTCUSD features: {len(preprocessor.feature_columns)}")

# Display feature columns
print("\nTechnical Indicators Added:")
for i, col in enumerate(preprocessor.feature_columns[:10]):
    print(f"{i+1}. {col}")
if len(preprocessor.feature_columns) > 10:
    print(f"... and {len(preprocessor.feature_columns) - 10} more")

## Model Training

Train LSTM models with attention mechanisms for both trading pairs.

In [ ]:
# Import training and model modules
from train import ModelTrainer
from lstm_model import create_model
import config

# Train XAUUSD model
print("Training XAUUSD LSTM model...")
xauusd_trainer = ModelTrainer('XAUUSD')
xauusd_model = create_model('attention', input_shape=(config.SEQUENCE_LENGTH, len(preprocessor.feature_columns)))
xauusd_history = xauusd_trainer.train(xauusd_model, xauusd_processed)

# Train BTCUSD model
print("\nTraining BTCUSD LSTM model...")
btcusd_trainer = ModelTrainer('BTCUSD')
btcusd_model = create_model('attention', input_shape=(config.SEQUENCE_LENGTH, len(preprocessor.feature_columns)))
btcusd_history = btcusd_trainer.train(btcusd_model, btcusd_processed)

print("\nTraining completed!")

## Training Results

Visualize the training performance.

In [ ]:
# Plot training history for XAUUSD
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(xauusd_history['train_loss'], label='Train Loss')
plt.plot(xauusd_history['val_loss'], label='Validation Loss')
plt.title('XAUUSD Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(xauusd_history['train_mae'], label='Train MAE')
plt.plot(xauusd_history['val_mae'], label='Validation MAE')
plt.title('XAUUSD Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

# Plot training history for BTCUSD
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(btcusd_history['train_loss'], label='Train Loss')
plt.plot(btcusd_history['val_loss'], label='Validation Loss')
plt.title('BTCUSD Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(btcusd_history['train_mae'], label='Train MAE')
plt.plot(btcusd_history['val_mae'], label='Validation MAE')
plt.title('BTCUSD Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

## Backtesting

Test the trained models on historical data to evaluate trading performance.

In [ ]:
# Import prediction module
from predict import TradingPredictor

# Initialize predictors
xauusd_predictor = TradingPredictor('XAUUSD')
btcusd_predictor = TradingPredictor('BTCUSD')

# Load the trained models
xauusd_predictor.load_model(xauusd_model, preprocessor)
btcusd_predictor.load_model(btcusd_model, preprocessor)

# Run backtests
print("Running XAUUSD backtest...")
xauusd_results = xauusd_predictor.backtest_strategy(xauusd_processed)

print("\nRunning BTCUSD backtest...")
btcusd_results = btcusd_predictor.backtest_strategy(btcusd_processed)

print("\nBacktesting completed!")

## Results and Analysis

Display the backtesting results and performance metrics.

In [ ]:
# Display XAUUSD results
print("=== XAUUSD Backtest Results ===")
for key, value in xauusd_results.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

print("\n=== BTCUSD Backtest Results ===")
for key, value in btcusd_results.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

# Plot equity curves
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(xauusd_results['equity_curve'])
plt.title('XAUUSD Equity Curve')
plt.xlabel('Trade')
plt.ylabel('Portfolio Value')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(btcusd_results['equity_curve'])
plt.title('BTCUSD Equity Curve')
plt.xlabel('Trade')
plt.ylabel('Portfolio Value')
plt.grid(True)

plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated a complete LSTM-based trading AI system that can:

1. **Collect Data**: Automatically scrape historical price data from Yahoo Finance
2. **Process Features**: Add 40+ technical indicators for comprehensive analysis
3. **Train Models**: Build and train LSTM networks with attention mechanisms
4. **Generate Predictions**: Make price predictions for trading decisions
5. **Backtest Strategies**: Evaluate performance with realistic trading simulation

### Key Features:
- **Multi-Asset Support**: Works with XAUUSD (Gold) and BTCUSD (Bitcoin)
- **Advanced Architecture**: LSTM with attention mechanisms for better predictions
- **Risk Management**: Stop-loss, take-profit, and position sizing
- **Comprehensive Analysis**: 40+ technical indicators and features
- **Easy Deployment**: Works in Google Colab with no local setup required

### Next Steps:
- Fine-tune hyperparameters for better performance
- Add more trading pairs and assets
- Implement live trading capabilities
- Add more advanced risk management features

**Disclaimer**: This is for educational purposes only. Past performance does not guarantee future results. Always test thoroughly before using in live trading.